In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mne.viz import plot_topomap

# === 1. Channel mapping and coordinates ===
channel_map = {
    "EEG 1": "Fp1", "EEG 2": "Fp2", "EEG 3": "F3", "EEG 4": "F4",
    "EEG 5": "C3", "EEG 6": "C4", "EEG 7": "P3", "EEG 8": "P4",
    "EEG 9": "O1", "EEG 10": "O2", "EEG 11": "F7", "EEG 12": "F8",
    "EEG 13": "T7", "EEG 14": "T8", "EEG 15": "P7", "EEG 16": "P8",
    "EEG 17": "Fz", "EEG 19": "Pz", "EEG 20": "Oz",
    "EEG 21": "Above M1", "EEG 22": "Above M2", "EEG 23": "P7-T7", "EEG 24": "P8-T8",
    "EEG 25": "P7-O1", "EEG 26": "P8-O2", "EEG 27": "Nasion-Fz", "EEG 28": "Cz-Fz"
}

# Load coordinates
tree = ET.parse("data/coordinates.xml")
root = tree.getroot()
ns = {"egi": "http://www.egi.com/coordinates_mff"}
scale_factor = 125

coords = []
for sensor in root.find("egi:sensorLayout", ns).find("egi:sensors", ns).findall("egi:sensor", ns):
    num = int(sensor.find("egi:number", ns).text)
    if 1 <= num <= 32:
        x = float(sensor.find("egi:x", ns).text) / scale_factor
        y = float(sensor.find("egi:y", ns).text) / scale_factor
        coords.append((f"EEG {num}", np.array([x, y])))

montage_pos = {name: pos for name, pos in coords}

# === 2. Load all 3 datasets ===
file_labels = [
    ("SP_ISA_S_all_viz.csv", "Mean ISA per Spindle"),
    ("SP_DENS_all_viz.csv", "Mean Spindle Density"),
    ("SP_FFT_all_viz.csv", "Mean Spindle Frequency")
]

# === 3. Create combined plot ===
fig, axes = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)
vmin, vmax = -0.6, 0.6  # fixed color scale for consistency

for ax, (fname, title) in zip(axes, file_labels):
    df = pd.read_csv(f"results_csv_out/{fname}")
    channel_names = df["EEG_label"].tolist()
    values = df["estimate"].to_numpy()
    sig_mask = df["Significant"].astype(bool)

    pos = np.array([montage_pos[ch] for ch in channel_names])
    display_names = [channel_map.get(ch, ch) for ch in channel_names]

    im, _ = plot_topomap(
        values,
        pos,
        axes=ax,
        show=False,
        names=None,
        cmap="RdBu_r",
        vlim=(vmin, vmax)
    )

    # Show only significant labels
    for i, (x, y) in enumerate(pos):
        if sig_mask[i]:
            ax.text(
                x, y,
                display_names[i],
                fontsize=10,
                ha='center', va='center'
            )

    ax.scatter(pos[sig_mask, 0], pos[sig_mask, 1],
               s=200, facecolors='yellow', edgecolors='black', linewidths=1)
    ax.set_title(title, fontsize=11)

# === 4. Shared colorbar below ===
cbar_ax = fig.add_axes([0.25, 0.08, 0.5, 0.03])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
cbar.set_label("Regression Beta (Age Effect)")

# === 5. Save and show ===
plt.savefig("mne_plots_out/spindle_age_all_combined.pdf", bbox_inches='tight')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import mne
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

# Load the dataset (with EEG X naming)
df_so = pd.read_csv("results_csv_out/spindle_features_num_ch.csv")

# Define channel mapping (10-20 to EEG X naming)
channel_map = {
    "Fp1": "EEG 1", "Fp2": "EEG 2", 
    "F3": "EEG 3", "F4": "EEG 4",
    "C3": "EEG 5", "C4": "EEG 6", 
    "P3": "EEG 7", "P4": "EEG 8",
    "O1": "EEG 9", "O2": "EEG 10",
    "F7": "EEG 11", "F8": "EEG 12",
    "T7": "EEG 13", "T8": "EEG 14", 
    "P7": "EEG 15", "P8": "EEG 16",
    # "Fz": "EEG 17",              # Mid-frontal
    # EEG 18 intentionally skipped (NA)
    # "Pz": "EEG 19", "Oz": "EEG 20",

    "above_M1": "EEG 21", "above_M2": "EEG 22",
    "between_P7_T7": "EEG 23", "between_P8_T8": "EEG 24",
    "between_P7_O1": "EEG 25", "between_P8_O2": "EEG 26",
    # "between_nasion_Fz": "EEG 27", "between_Cz_Fz": "EEG 28",

    # Optional placeholders for EEG 18, 29–32 if ever used or required
    # "EEG 18": None,
    # "EEG 29": None,
    # "EEG 30": None,
    # "EEG 31": None,
    # "EEG 32": None
}

# Inverse map: EEG X -> 10-20 name (e.g., "EEG 12" → "F8")
inverse_channel_map = {v: k for k, v in channel_map.items()}

# Parse the XML file explicitly
xml_path = "data/coordinates.xml"
tree = ET.parse(xml_path)
root = tree.getroot()
ns = {'ns': 'http://www.egi.com/coordinates_mff'}

scale_factor = 175

# Extract positions explicitly and scale by 200, only for channels present in data
ch_pos = {}
for sensor in root.findall('.//ns:sensor', ns):
    num = sensor.find('ns:number', ns).text
    ch_name = f'EEG {num}'
    if ch_name in channel_map.values():
        x = float(sensor.find('ns:x', ns).text) / scale_factor
        y = float(sensor.find('ns:y', ns).text) / scale_factor
        z = float(sensor.find('ns:z', ns).text) / scale_factor
        ch_pos[ch_name] = np.array([x, y, z])

# Verify correct extraction
print("Extracted channels:", ch_pos.keys())

# Explicitly create info and montage with only relevant channels
info = mne.create_info(ch_names=list(ch_pos.keys()), sfreq=250, ch_types='eeg')
montage_custom = mne.channels.make_dig_montage(ch_pos=ch_pos, coord_frame='head')
info.set_montage(montage_custom)

# Extract columns matching spindle relative phase for EEG channels
sp_cols = [f'SP_R_PHASE_IF_all_{ch}' for ch in ch_pos.keys()]
group_means = df_so.groupby(['group', 'sex'])[sp_cols].mean()

# Plotting setup explicitly
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
titles = ['TD Male', 'ASD Male', 'TD Female', 'ASD Female']
group_sex_pairs = [('HC', 'M'), ('ASD', 'M'), ('HC', 'F'), ('ASD', 'F')]

# Highlight EEG 12 (F8 equivalent)
highlight_channel = 'EEG 12'
mask = np.array([ch == highlight_channel for ch in info.ch_names])
mask_params = dict(marker='o', markerfacecolor='yellow', markeredgecolor='black',
                   linewidth=1, markeredgewidth=1, markersize=12)

# Iterate explicitly to plot data
for ax, (group, sex), title in zip(axes, group_sex_pairs, titles):
    data_plot = [group_means.loc[group, sex][f'SP_R_PHASE_IF_all_{ch}'] for ch in info.ch_names]

    # Topomap plotting with explicit data
    im, _ = mne.viz.plot_topomap(
        data_plot, info, axes=ax, cmap='RdBu_r',
        vlim=(np.nanmin(group_means.values), np.nanmax(group_means.values)),
        sensors=True, show=False, mask=mask, mask_params=mask_params,
        names=[inverse_channel_map.get(ch, ch) for ch in info.ch_names]
    )

    ax.set_title(title, fontsize=12, fontweight='bold')

# Add colorbar explicitly
cbar = fig.colorbar(im, ax=axes, orientation='horizontal', fraction=0.05, pad=0.1)
cbar.set_label('Mean SO-Phase-dependent Spindle Frequency Modulation', fontsize=12)

# Save and show explicitly
fig.savefig("mne_plots_out/SP_R_PHASE_IF_all_F8.pdf", bbox_inches='tight')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import mne
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

# 1. Load the ISO relative power data
df_iso = pd.read_csv("results_csv_out/iso_relative_power.csv")

# 2. Define mapping from 10–20 labels to EEG channel numbers (scalp only)
channel_map = {
    "Fp1": "EEG 1", "Fp2": "EEG 2", "F3": "EEG 3", "F4": "EEG 4",
    "C3": "EEG 5", "C4": "EEG 6", "P3": "EEG 7", "P4": "EEG 8",
    "O1": "EEG 9", "O2": "EEG 10", "F7": "EEG 11", "F8": "EEG 12",
    "T7": "EEG 13", "T8": "EEG 14", "P7": "EEG 15", "P8": "EEG 16",
    "Fz": "EEG 17", "Pz": "EEG 19", "Oz": "EEG 20",
    "Above M1": "EEG 21", "Above M2": "EEG 22",
    "P7-T7": "EEG 23", "P8-T8": "EEG 24",
    "P7-O1": "EEG 25", "P8-O2": "EEG 26",
    "Nasion-Fz": "EEG 27", "Cz-Fz": "EEG 28"
}
inverse_channel_map = {v: k for k, v in channel_map.items()}

# 3. Parse XML file for coordinates
tree = ET.parse('data/coordinates.xml')
root = tree.getroot()
ns = {'ns': 'http://www.egi.com/coordinates_mff'}

# Extract only scalp channels' positions
scale_factor = 175

ch_pos = {}
for sensor in root.findall('.//ns:sensor', ns):
    num = sensor.find('ns:number', ns).text
    name = f'EEG {num}'
    if name in inverse_channel_map:  # only scalp channels
        x = float(sensor.find('ns:x', ns).text) / scale_factor
        y = float(sensor.find('ns:y', ns).text) / scale_factor
        z = float(sensor.find('ns:z', ns).text) / scale_factor
        ch_pos[name] = np.array([x, y, z])

# 4. Create montage and info using only scalp channels
eeg_channels = list(ch_pos.keys())
info = mne.create_info(ch_names=eeg_channels, sfreq=250, ch_types='eeg')
montage = mne.channels.make_dig_montage(ch_pos=ch_pos, coord_frame='head')
info.set_montage(montage)

# 5. Extract mean values only for scalp ISO channels
iso_cols = [f"{ch}_relative_ISO_power_0.005-0.03Hz" for ch in eeg_channels]
group_means_iso = df_iso.groupby(['group', 'sex'])[iso_cols].mean()

# 6. Plotting
fig, axes = plt.subplots(1, 4, figsize=(18, 4))
titles = ['TD Male', 'ASD Male', 'TD Female', 'ASD Female']
group_sex_pairs = [('HC', 'M'), ('ASD', 'M'), ('HC', 'F'), ('ASD', 'F')]

vmin = group_means_iso.min().min()
vmax = group_means_iso.max().max()

for ax, (group, sex), title in zip(axes, group_sex_pairs, titles):
    data_plot = []
    for ch in eeg_channels:
        col_name = f'{ch}_relative_ISO_power_0.005-0.03Hz'
        value = group_means_iso.loc[group, sex][col_name]
        data_plot.append(value)
    data_plot = np.array(data_plot)

    im, _ = mne.viz.plot_topomap(
        data_plot,
        info,
        axes=ax,
        cmap='RdBu_r',
        vlim=(vmin, vmax),
        sensors=True,
        show=False,
        #names=[inverse_channel_map.get(ch, ch) for ch in eeg_channels]
    )
    ax.set_title(title, fontsize=12, fontweight='bold')

# 7. Colorbar and export
cbar = fig.colorbar(im, ax=axes, orientation='horizontal', fraction=0.05, pad=0.1)
cbar.set_label('Mean Relative ISO Band Power (%)', fontsize=12)

fig.savefig("mne_plots_out/relative_power_scalp_only_labeled.pdf", bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import mne
import xml.etree.ElementTree as ET

# === Load correlation results ===
df = pd.read_csv("results_csv_out/iso_corr_df.csv")

# === Map EEG 10-20 names to "EEG X" ===
channel_map = {
    "Fp1": "EEG 1", "Fp2": "EEG 2", "F3": "EEG 3", "F4": "EEG 4",
    "C3": "EEG 5", "C4": "EEG 6", "P3": "EEG 7", "P4": "EEG 8",
    "O1": "EEG 9", "O2": "EEG 10", "F7": "EEG 11", "F8": "EEG 12",
    "T7": "EEG 13", "T8": "EEG 14", "P7": "EEG 15", "P8": "EEG 16",
    "Fz": "EEG 17", "Pz": "EEG 19", "Oz": "EEG 20",
    "Above M1": "EEG 21", "Above M2": "EEG 22",
    "P7-T7": "EEG 23", "P8-T8": "EEG 24",
    "P7-O1": "EEG 25", "P8-O2": "EEG 26",
    "Nasion-Fz": "EEG 27", "Cz-Fz": "EEG 28"
}
inverse_map = {v: k for k, v in channel_map.items()}

# === Parse channel positions ===
tree = ET.parse("data/coordinates.xml")
root = tree.getroot()
ns = {"ns": "http://www.egi.com/coordinates_mff"}

scale_factor = 125

ch_pos = {}
for sensor in root.findall('.//ns:sensor', ns):
    num = int(sensor.find('ns:number', ns).text)
    name = f"EEG {num}"
    if name in inverse_map:  # Only keep mapped channels
        x = float(sensor.find('ns:x', ns).text) / scale_factor
        y = float(sensor.find('ns:y', ns).text) / scale_factor
        z = float(sensor.find('ns:z', ns).text) / scale_factor
        ch_pos[name] = np.array([x, y, z])

# === Set montage and info ===
info = mne.create_info(ch_names=list(ch_pos.keys()), sfreq=250, ch_types='eeg')
montage = mne.channels.make_dig_montage(ch_pos=ch_pos, coord_frame="head")
info.set_montage(montage)

# === Prepare figure layout ===
clinical_vars = df["Clinical"].unique()
n_cols = 5
n_rows = int(np.ceil(len(clinical_vars) / n_cols))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 3.2, n_rows * 3), squeeze=False)

# === Plot each topomap ===
for i, clinical_var in enumerate(clinical_vars):
    ax = axes[i // n_cols, i % n_cols]
    sub_df = df[df["Clinical"] == clinical_var].copy()

    # Map to EEG numbers and match with positions
    sub_df["EEG_label"] = sub_df["Channel_1020"].map(channel_map)
    sub_df = sub_df.dropna(subset=["EEG_label"])
    ch_labels = sub_df["EEG_label"].tolist()
    corr_vals = sub_df["Correlation"].values
    sig_mask = sub_df["p_value"].values < 0.05

    pos = np.array([ch_pos[ch][:2] for ch in ch_labels])
    names = [inverse_map[ch] for ch in ch_labels]

    im, _ = mne.viz.plot_topomap(
        corr_vals, pos, axes=ax, show=False, cmap="RdBu_r", vlim=(-1, 1),
        sensors=True,
        mask=sig_mask,
        mask_params=dict(marker='o', markerfacecolor='yellow', markeredgecolor='black',
                         linewidth=1, markersize=12)
    )

    # Add labels only for significant channels
    for (x, y), name, sig in zip(pos, names, sig_mask):
        if sig:
            ax.text(x, y, name, fontsize=7, ha='center', va='center')

    ax.set_title(clinical_var, fontsize=10)

# === Remove empty plots ===
for j in range(i + 1, n_rows * n_cols):
    fig.delaxes(axes[j // n_cols, j % n_cols])

# === Shared colorbar ===
cbar_ax = fig.add_axes([0.25, 0.08, 0.5, 0.03])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', label="Pearson Correlation (r)")

fig.savefig("mne_plots_out/corr_iso_plot.pdf", bbox_inches='tight')
plt.show()